In [ ]:
pip install pyyaml


In [ ]:
!pip uninstall pipenv
!pip install pipenv

In [2]:
pip install vaex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 3.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/opt/python/latest/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import vaex as vx
from filters.filters import FilterCreation
from filter_values.vaex_filtes import LoadFilterValues

# load data frame

In [4]:
data_frame = vx.from_csv("./dataset/processed_crimes_sample_5000.csv")

In [5]:
# print head
data_frame.head(5)

#,Year,Date,ID,Case Number,Date.1,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year.1,Updated On,Latitude,Longitude,Location,Datetime,Day,Time,Month
0,2001,2001-12-28 19:00:00,1918721,G775843,2001-12-28,027XX W JACKSON BLVD,5001,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,STREET,True,False,1125,11,2,27,26,1.15839e+06,1.89862e+06,2001,08/17/2015 03:03:40 PM,41.8776,-87.6939,"(41.877569562, -87.693904749)",2001-12-28 19:00:00,Friday,19:00:00,12
1,2001,2001-10-16 00:01:00,2123198,HH358469,2001-10-16,071XX W GRAND AVE,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,OTHER,False,True,2512,25,36,18,11,1.12785e+06,1.91526e+06,2001,08/17/2015 03:03:40 PM,41.9238,-87.8057,"(41.923801639, -87.805661998)",2001-10-16 00:01:00,Tuesday,00:01:00,10
2,2001,2001-10-28 12:00:00,2100699,HH332385,2001-10-28,065XX S FRANCISCO AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,831,8,15,66,14,1.15822e+06,1.86092e+06,2001,08/17/2015 03:03:40 PM,41.7741,-87.6956,"(41.774097713, -87.695556749)",2001-10-28 12:00:00,Sunday,12:00:00,10
3,2001,2001-05-01 00:00:00,3164428,HK164197,2001-05-01,042XX W HARRISON ST,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,1132,11,24,26,06,1.14788e+06,1.89705e+06,2001,03/31/2006 10:03:38 PM,41.8735,-87.7325,"(41.873464135, -87.732524372)",2001-05-01 00:00:00,Tuesday,00:00:00,5
4,2001,2001-01-11 04:30:00,648,G021349,2001-01-11,071XX S BENNETT AVE,0110,HOMICIDE,FIRST DEGREE MURDER,APARTMENT,True,True,333,3,5,43,01A,1.18994e+06,1.85799e+06,2001,03/19/2019 04:11:22 PM,41.7654,-87.5794,"(41.765361316, -87.579354115)",2001-01-11 04:30:00,Thursday,04:30:00,1


In [6]:
filters = FilterCreation(data_frame)


In [7]:
df_slice_obj = LoadFilterValues(data_frame)
df_slice_obj.get_days_of_week()




['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [8]:
data_frame.dtypes


Year                      int64
Date                     string
ID                        int64
Case Number              string
Date.1                   string
Block                    string
IUCR                     string
Primary Type             string
Description              string
Location Description     string
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 string
X Coordinate            float64
Y Coordinate            float64
Year.1                    int64
Updated On               string
Latitude                float64
Longitude               float64
Location                 string
Datetime                 string
Day                      string
Time                     string
Month                     int64
dtype: object

In [19]:
# create selection filter


years=[2010,2011,2012,2013,2014,2015,2016,2017,2018]
months=[1,2,3,4,5,6,7,8,9,10,11,12]
types=["THEFT","ASSAULT","NARCOTICS","BATTERY","CRIMINAL DAMAGE","OTHER OFFENSE","ROBBERY"]
districts= [1,2,3,5,10]

result =data_frame.select(data_frame.Month.isin([12]), mode='and')
result= data_frame.select(data_frame["Year"].isin([2001,2002,2003]), mode='and')
result = data_frame.select(data_frame.District.isin(districts), mode='and')

result = data_frame.select(data_frame.Month.isin(months), mode='and')
result = data_frame.select(data_frame["Primary Type"].isin(types), mode='and')

In [23]:
# get years filter
years = data_frame["Year"].unique() # get unique years
years[:5]

[2009, 2003, 2004, 2005, 2006]

In [24]:
# get_crime_types
types = data_frame["Primary Type"].unique() # get unique crime types
types[:5]

['LIQUOR LAW VIOLATION',
 'OTHER OFFENSE',
 'ARSON',
 'OTHER NARCOTIC VIOLATION',
 'THEFT']